In [1]:
import tensorflow.compat.v1 as tf 
tf.disable_v2_behavior()
import random

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
tf.set_random_seed(777)

In [3]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

In [5]:
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

In [6]:
# dropout 하기 위한 변수 
keep_prob = tf.placeholder(tf.float32)

In [7]:
W1 = tf.get_variable("W1", shape=[784, 512], initializer= tf.keras.initializers.glorot_normal())
b1 = tf.Variable(tf.random_normal([512]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
# dropout 적용 
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

W2 = tf.get_variable("W2", shape=[512, 512], initializer= tf.keras.initializers.glorot_normal())
b2 = tf.Variable(tf.random_normal([512]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
# dropout 적용 
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

W3 = tf.get_variable("W3", shape=[512, 512], initializer= tf.keras.initializers.glorot_normal())
b3 = tf.Variable(tf.random_normal([512]))
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)
# dropout 적용 
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)

W4 = tf.get_variable("W4", shape=[512, 512], initializer= tf.keras.initializers.glorot_normal())
b4 = tf.Variable(tf.random_normal([512]))
L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
# dropout 적용 
L4 = tf.nn.dropout(L3, keep_prob=keep_prob)

W5 = tf.get_variable("W5", shape=[512, 10], initializer= tf.keras.initializers.glorot_normal())
b5 = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L4, W5) + b5

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [8]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)     

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [9]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [10]:
for epoch in range(training_epochs):
    avg_cost = 0
    
    for i in range(total_batch):
        # batch size 만큼 x, y dataset 할당
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)        
        # 학습할때는 0.7로, test할때는 1로
        feed_dict = {X:batch_xs, Y:batch_ys, keep_prob:0.7}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch
        
    print('Epoch: ', '%04d' % (epoch +1), 'cost= ', '{:.9f}'.format(avg_cost))
    
print('Learning finished!-----------')

Epoch:  0001 cost=  0.468945044
Epoch:  0002 cost=  0.164668045
Epoch:  0003 cost=  0.125138372
Epoch:  0004 cost=  0.104233122
Epoch:  0005 cost=  0.087392651
Epoch:  0006 cost=  0.078018915
Epoch:  0007 cost=  0.069824041
Epoch:  0008 cost=  0.066471662
Epoch:  0009 cost=  0.058031424
Epoch:  0010 cost=  0.056841923
Epoch:  0011 cost=  0.052689956
Epoch:  0012 cost=  0.049749160
Epoch:  0013 cost=  0.044884059
Epoch:  0014 cost=  0.041430055
Epoch:  0015 cost=  0.041820633
Learning finished!-----------


In [11]:
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy: ', sess.run(accuracy, feed_dict={
            X:mnist.test.images, Y: mnist.test.labels, keep_prob:1
}))


Accuracy:  0.9832


dropout을 적용한 결과, deep 한것보다 정확도가 높게 나왔다. 

In [14]:
r = random.randint(0, mnist.test.num_examples -1)
print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r+1], 1)))
print("Prediction: ", sess.run(tf.argmax(hypothesis, 1), feed_dict={X:mnist.test.images[r:r+1], keep_prob:1}
                              ))

Label:  [6]
Prediction:  [6]


예측한 값도 잘 나왔다.